In [ ]:
import pandas as pd
import hashlib
import re
from cryptography.fernet import Fernet

file_path = "./Laptop_price.csv"
try:
    df = pd.read_csv(file_path)
except Exception as e:
    print(f"Ошибка загрузки файла: {e}")

def check_scv_injection(df):
        dangerous_chars = ('=', '+', '-', '@')
        for col in df.select_dtypes(include=['object']). columns:
            if df[col].astype(str).apply(lambda x: x.startswith(dangerous_chars) or has_space_before_dangerous_char(x, dangerous_chars)).any():
                print(f"Обнаружены потенциальные CSV-инъекции в столбце {col}!")
            else:
                print(f"Столбец {col} безопасен.")

# Функция проверки на пробел перед опасным символом
def has_space_before_dangerous_char(value, dangerous_chars):
    for char in dangerous_chars:
        if char in value:
            idx = value.find(char)
            if idx > 0 and value[idx - 1] == ' ':  # Проверяем наличие пробела перед символом
                return True
            return False

check_scv_injection(df)

def clean_input(value):
    sql_keywords = ["SELECT", "DROP", "DELETE", "INSERT", "UPDATE", "ALTER", "UNION"]
    xss_patterns = [r'<script.*?</script>', r'javascript:.*', r'onerror=.*']

    # Проверка на наличие SQL комментариев (--)
    if '--' in value:
        return "[BLOCKED]"

    # Проверка на наличие SQL ключевых слов
    for keyword in sql_keywords:
        if keyword.lower() in value.lower():
            return "[BLOCKED]"
        
    # Проверка на наличие XSS паттернов
    for pattern in xss_patterns:
        if re.search(pattern, value, re.IGNORECASE):
            return "[BLOCKED]"
        
    return value

df = df.applymap(lambda x: clean_input(str(x)) if isinstance(x, str) else x)
print("Фильтрация данных завершена.")

# Функция для хеширования
def hash_parameter(parametr):
    return hashlib.sha256(str(parametr).encode()).hexdigest()

df['Price_Hashed'] = df['Price'].apply(hash_parameter)
print("Столбец с хешированными ценами добавлен.")

df['Brand_Hashed'] = df['Brand'].apply(hash_parameter)
print("Столбец с хешированными брендами добавлен.")

# Генерация ключа для шифрования и дешифрования
кеу = Fernet.generate_key()
cipher = Fernet(кеу)

# Функция для шифрования параметров
def encrypt_parameter(parameter):
    return cipher.encrypt(str(parameter).encode()).decode()

# Функция для расшифровки
def decrypt_parameter(encrypted_parameter):
    return cipher.decrypt(encrypted_parameter.encode()).decode()

df['Price_Encrypted'] = df['Price'].apply(encrypt_parameter)
print("Столбец с зашифрованными ценами добавлен.")

df['Brand_Encrypted'] = df['Brand'].apply(encrypt_parameter)
print("Столбец с зашифрованными брендами добавлен.")

output_path = "./Laptop_price_secured.csv"
df.to_csv(output_path, index=False)
print(f"Обработанный файл сохранен: {output_path}")

# Функция для вывода расшифрованных примеров
def show_decrypted_samples(column_name='Price_Encrypted', count=5):
    decrypted_samples = []
    for val in df[column_name].head(count):
        try:
            decrypted = decrypt_parameter(val)
            decrypted_samples.append(decrypted)
        except Exception as e:
            decrypted_samples.append(f"[Ошибка: {e}]")
    return decrypted_samples

print(show_decrypted_samples())

Столбец Brand безопасен.
Фильтрация данных завершена.
Столбец с хешированными ценами добавлен.
Столбец с хешированными брендами добавлен.
Столбец с зашифрованными ценами добавлен.
Столбец с зашифрованными брендами добавлен.
Обработанный файл сохранен: ./Laptop_price_secured.csv
['17395.093064560457', '31607.60591926617', '9291.023542338657', '17436.72833420527', '32917.990718167515']


C:\Users\79778\AppData\Local\Temp\ipykernel_12124\3907803641.py:51: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: clean_input(str(x)) if isinstance(x, str) else x)
